In [ ]:
!mkdir -p project
!touch project/__init__.py

# Tests unitarios

Los tests unitarios (o unit tests) son la forma más común de tests. En pocas palabras, estos actúan sobre las partes más pequeñas y testables de una aplicación para asegurarse de que esa funcionalidad en particular funciona correctamente. Esto implica recorrer las partes más pequeñas e individuales del código. Idealmente, se realiza una prueba unitaria para cada parte del código en la aplicación, de forma aislada. Las pruebas unitarias son únicas en que son:

Muy rápidas: en milisegundos o incluso más rápidas.
Aisladas de dependencias lentas y/o propensas a errores (llamados a la DB o a APIs, etc)
Deterministas: Si bien nos gustaría que todas las pruebas fueran deterministas, lograrlo para pruebas de integración o de end to end, es difícil en la práctica. Este criterio es de vital importancia para las pruebas unitarias.


## Modulo python unittest

El módulo unittest es un framework de pruebas unitarias integrado en Python. Proporciona una serie de clases y métodos que permiten escribir y ejecutar pruebas unitarias de manera eficiente y organizada. Este está inspirado en otros frameworks de testeo unitario, como JUnit en Java. Permite a los desarrolladores crear casos de prueba, agruparlos en suites de pruebas y ejecutarlos de manera automatizada.

In [ ]:
# %%writefile project/person.py
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    surname: str
    title: str

    def set_name(self, user_name):
        self.name = user_name
        return self.name

    def get_name(self):
        if not self.name and not self.surname:
            raise Exception('Esta persona no tiene nombre')
        else:
            return f"{self.surname}, {self.name}"

In [ ]:
# %%writefile project/test_person.py
# from .person import Person
import unittest
import sys

class TestPerson(unittest.TestCase):
    def setUp(self):
        print("Preparando la prueba")
        self.test_person = Person("Nachito", "Villaluenga", "Ingeniero de Software")

    def tearDown(self):
        print("terminando la prueba")
        del self.test_person

    def test_set_name(self):
        # Arrange
        print("Testeando test_set_name")
        persona = self.test_person
        nombre_esperado = "Jose Ignacio"

        # Act
        nombre_obtenido = persona.set_name(nombre_esperado)

        # Assert
        self.assertEqual(nombre_esperado, nombre_obtenido, "Los nombres no son iguales")

    def test_get_name(self):
        # Arrange
        print("Testeando test_get_name")
        persona = self.test_person
        nombre_esperado = "Villaluenga, Nachito"

        # Act
        nombre_obtenido = persona.get_name()

        # Assert
        self.assertEqual(nombre_esperado, nombre_obtenido, "Los nombres no son iguales")

    @unittest.skip("Salteamos este test")
    def test_nothing(self):
        self.fail("Nunca va a llegar aca porque skipeamos")

    @unittest.skipIf(not sys.platform.startswith("win"),
                     "Solo testeamos en windows")
    def test_format(self):
        # Tests that work for only a certain version of the library.
        pass


In [ ]:
unittest.main(argv=[''], verbosity=2, exit=False)

### Arrange-Act-Assert Pattern

es una excelente manera de estructurar casos de prueba. Prescribe un orden de operaciones:

1. Arrange: organiza las entradas y los objetivos, aqui se debe configurar el caso de prueba. ¿Requiere el test algún objeto o configuración especial? ¿Es necesario preparar una base de datos? ¿Es necesario iniciar sesión en una aplicación web? Realizamos todas estas operaciones aqui al principio de los tests.

2. Act: Actuaa sobre el comportamiento que queremos probar, los pasos para cubrir lo que se va a probar. Esto podría ser llamar a una función o método, llamar a una API REST o interactuar con una página web, debemos mantener las acciones enfocadas en el comportamiento que queremos probar.

3. Assert, probamos los resultados esperados, aqui debemos generar algún tipo de respuesta. Los pasos del assert verifican si esa respuesta es buena o mala, a veces, las afirmaciones son tan simples como verificar valores numéricos o de una lista, otras veces, pueden requerir verificar múltiples aspectos del sistema. Los asserts finalmente determinarán si el test aprueba o falla.

## Modulo python pytest

Pytest es un framework de testeo de Python que se originó a partir del proyecto PyPy. Se puede usar para escribir varios tipos de pruebas de software, incluidas pruebas unitarias, pruebas de integración, pruebas de extremo a extremo y mas.

In [ ]:
!pip install pytest pytest-cov

In [ ]:
# %%writefile project/test_person.py
# from .person import Person
import pytest
import sys

@pytest.fixture
def get_person():
    print("Preparando la prueba")
    yield Person("Nachito", "Villaluenga", "Ingeniero de Software")
    print("terminando la prueba")


def test_set_name(get_person):
    # Arrange
    print("Testeando test_set_name")
    nombre_esperado = "Jose Ignacio"

    # Act
    nombre_obtenido = get_person.set_name(nombre_esperado)

    # Assert
    assert nombre_esperado == nombre_obtenido, "Los nombres no son iguales"

def test_get_name(get_person):
    # Arrange
    print("Testeando test_get_name")
    nombre_esperado = "Villaluenga, Nachito"

    # Act
    nombre_obtenido = get_person.get_name()

    # Assert
    assert nombre_esperado == nombre_obtenido, "Los nombres no son iguales"

@pytest.mark.skip("Salteamos este test")
def test_nothing():
    raise Exception("Nunca va a llegar aca porque skipeamo")

@pytest.mark.skipIf(not sys.platform.startswith("win"),
                    "Solo testeamos en windows")
def test_format():
    # Tests that work for only a certain version of the library.
    pass


In [ ]:
!pytest --cov project/

In [ ]:
!pytest --cov --cov-report=html:coverage_re
!tar -cvf coverage.tar coverage_re

In [ ]:
# %%writefile project/person_functions.py
# from .person import Person
import requests

def create_person_with_input():
    name = input("Escribre el nombre de la persona: ")
    surname = input("Escribre el apellido de la persona: ")
    title = input("Escribre el titulo de la persona: ")
    return Person(name, surname, title)

def tell_a_joke(person: Person):
    result = requests.get("https://v2.jokeapi.dev/joke/Programming?lang=es&type=single")
    json = result.json()
    if json["type"] == "single":
        joke = json["joke"]
    else:
        joke = f'{json["setup"]}\n{json["delivery"]}'
    print(f"{person.get_name()} dice una broma:")
    print(joke)

In [ ]:
person = create_person_with_input()
tell_a_joke(person)

In [ ]:
# %%writefile project/test_person_functions.py
# from .person_functions import create_person_with_input, tell_a_joke
# from .person import Person
from pytest import MonkeyPatch
import pytest
import sys
import requests

def test_create_person_with_input(monkeypatch: MonkeyPatch):
    # Arrange
    inputs = ["Jose Ignacio", "Villaluenga", "Ingeniero de software"]
    monkeypatch.setattr("builtins.input", lambda _: inputs.pop(0))
    nombre = "Jose Ignacio"
    apellido = "Villaluenga"
    titulo = "Ingeniero de software"

    # Act
    persona_obtenida = create_person_with_input()

    # Assert
    assert nombre == persona_obtenida.name
    assert apellido == persona_obtenida.surname
    assert titulo == persona_obtenida.title

class MockResponse:
    def __init__(self, json):
        self.json_data = json

    def json(self):
        return self.json_data

def test_tell_a_joke():
    # Arrange
    mock_response = MockClient({"type": "single", "joke": "mock jock"})
    monkeypatch.setattr(requests, "get", lambda _: mock_response)

    # Act
    tell_a_joke(Person("Nombre", "Apelldio", "titulo"), mock_response)

    # Assert

def test_tell_a_joke_fail(monkeypatch: MonkeyPatch):
    # Arrange
    mock_response = MockResponse({"type": "fail", "joke": "mock jock"})
    monkeypatch.setattr(requests, "get", lambda _: mock_response)

    # Act and assert
    with pytest.raises(KeyError):
        tell_a_joke(Person("Nombre", "Apelldio", "titulo"))


In [ ]:
!pytest --cov project/